In [1]:
# Solc 컴파일러
!pip install py-solc-x

In [11]:
from web3 import Web3, AsyncWeb3


# HTTP Provider를 통해 접속 가능.
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))

print(w3.is_connected())

True


In [28]:
from solcx import install_solc, compile_standard
install_solc(version='0.7.0')

/opt/anaconda3/envs/SmartSecurity/lib/python3.11/site-packages/solcx/install.py:714: UnexpectedVersionWarning: Installed solc version is v0.7.0+commit.9e61f92b, expecting v0.7.0
  warnings.warn(


<Version('0.7.0')>

In [29]:
from solcx import compile_source

## Original

In [30]:
compiled_sol = compile_source(
    '''
    // SPDX-License-Identifier: GPL-3.0
    pragma solidity >0.5.0;

    contract Greeter {
        string public greeting;

        constructor() {
            greeting = 'Hello';
        }

        function setGreeting(string memory _greeting) public {
            greeting = _greeting;
        }

        function greet() view public returns (string memory) {
            return greeting;
        }
    }
    ''',
    output_values=['abi', 'bin'],
    solc_version="0.7.0",
)

## GPT FIXED

In [ ]:
# 컴파일할 Solidity 소스 코드
sol_source = '''
// SPDX-License-Identifier: GPL-3.0
pragma solidity >0.5.0;

contract Greeter {
    string public greeting;

    constructor() public {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string memory) {
        return greeting;
    }
}
'''

# 컴파일 옵션 설정
compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {
        "Greeter.sol": {
            "content": sol_source
        }
    },
    "settings": {
        "outputSelection": {
            "*": {
                "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
            }
        }
    }
},
solc_version="0.8.24",  # 이 부분은 설치된 solc 버전에 맞게 조정할 수 있습니다.
)

## Original

In [31]:
contract_id, contract_interface = compiled_sol.popitem()

In [32]:
bytecode = contract_interface['bin']
abi = contract_interface['abi']

## GPT Fixed

In [ ]:
bytecode = compiled_sol['contracts']['Greeter.sol']['Greeter']['evm']['bytecode']

In [ ]:
abi = compiled_sol['contracts']['Greeter.sol']['Greeter']['abi']

## Deploy Contract

In [33]:
w3.eth.default_account = w3.eth.accounts[0]

In [34]:
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)

In [35]:
tx_hash = Greeter.constructor().transact()

In [41]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)

AttributeDict({'transactionHash': HexBytes('0xe063edb9dc7ef0c22c712df82a0a940df32020753a5ed348c10af0a981b047bf'), 'transactionIndex': 0, 'blockNumber': 3, 'blockHash': HexBytes('0x8421c53e9beeed12d7a7e68eee287c9ba537aa0839574d61624abbb76bfa5320'), 'from': '0xa99B9e4Cfcb7E3933F2265605E1DB6C394486784', 'to': '0xE23Ab52110ef0EE9872d358bdf5471dd8DA7bE3A', 'cumulativeGasUsed': 29688, 'gasUsed': 29688, 'contractAddress': None, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'effectiv

In [37]:
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

In [38]:
greeter.functions.greet().call()

'Hello'

In [40]:
tx_hash = greeter.functions.setGreeting('Arad4228').transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
greeter.functions.greet().call()

'Arad4228'